In [1]:
#Import functions
import polars as pl
from functools import partial
from itertools import product, combinations
import multiprocessing
from numba import jit
import numpy as np
import pickle

In [2]:
#Load dataframe from CSV
currentDf=pl.read_csv("../Data/SP500/minuteHist2021/tradingHours/AAPL.csv", infer_schema_length=None)

In [3]:
#Split into time segments
weekDfs2 = []
for k in range(currentDf['time'][0], currentDf['time'][-1], 1209600000):
    current=k
    tempDf=currentDf.filter((pl.col("time")) >= current)
    tempDf2=tempDf.filter((pl.col("time")) < current+1209600000)
    weekDfs2.append(tempDf2)
len(weekDfs2)

#Split into 1 week segments
weekDfs1 = []
for k in range(currentDf['time'][0], currentDf['time'][-1], 604800000):
    current=k
    tempDf=currentDf.filter((pl.col("time")) >= current)
    tempDf=tempDf.filter((pl.col("time")) < current+604800000)
    weekDfs1.append(tempDf)
len(weekDfs1)

#Split into 1 day segments
dayDfs1 = []
for k in range(currentDf['time'][0], currentDf['time'][-1], 86400000):
    current=k
    tempDf=currentDf.filter((pl.col("time")) >= current)
    tempDf=tempDf.filter((pl.col("time")) < current+86400000)
    dayDfs1.append(tempDf)
len(dayDfs1)

964

In [4]:
#Algo
#Numba for speed
@jit(nopython=True)
#Takes in list of paramaters and an array with all the data
def algo(paramsList, dataArray):
    fee=paramsList[0][0] #Percent change in price to look for
    timeout=int(paramsList[0][1]) #The timeout when it takes too long
    divisor=int(paramsList[0][2]) #Divisor for ema
    allParams=paramsList[1:]
    openPrices=dataArray[:,0] #Open prices
    
    #params has an array of arrays with ticks before, change, version
    
    tradePercent=0
    totalReturn=1

    total=0

    start=60 #Start of data
    end=int(len(openPrices)-61) #End of data

    for j in range(start, end):

        #Check that its not within the first or last hour of trading
        if 0<dataArray[j][2]-dataArray[j-60][2]<4500000 and 0<dataArray[j+60][2]-dataArray[j][2]<4500000:
        
        #Check that its all within one day of trading
        #if 0<dataArray[j][2]-dataArray[j-(gap*len(params))][2]<(60000*gap*len(params)*1.25) and 0<dataArray[j+timeout][2]-dataArray[j][2]<(60000*timeout*1.25):

        #Check that its all within first or last hour of trading
        #if 4500000<dataArray[j][2]-dataArray[j-60][2] or 4500000<dataArray[j+60][2]-dataArray[j][2]:

            #Set start price and continue
            startPrice=dataArray[j][0]
            continueParams=True

            for param in allParams:
                ticksBefore = int(param[0])
                change = param[1]
                version = param[2]

                #Set price we are checking
                if version==1:
                    checkPrice=dataArray[j-ticksBefore][0]


                elif version==2:

                    checkTotal = 0
                    #Exponential average by using equation of (1/2 * 1/2^n * price) (a/1-r) (1/3 * 2/3^n * price) which = 1 as it tends to infinity
                    expoError=0
                    divisor=4

                    count=0
                    for k in range(j-ticksBefore+1,j+1):
                        checkTotal+=(int(dataArray[k][0]) * (1/divisor * (1-(1/divisor))**(ticksBefore-count)))
                        count+=1

                    count=0
                    for k in range(j-ticksBefore+1,j+1):
                        expoError+=(1/divisor * (1-(1/divisor))**(ticksBefore-count))
                        count+=1

                    checkPrice = checkTotal/expoError


                elif version==3:
                    #checkPrice=sma(ticksBefore, j, openPrices)
                    checkTotal = 0
                    #Simple mean by add and divide
                    for k in range(j-ticksBefore+1,j+1):
                        checkTotal+=int(dataArray[k][0])
                    checkPrice = checkTotal/ticksBefore


                #If price now is above price we are checking, then continue
                if not checkPrice*(change+0.0002)>startPrice>checkPrice*change:
                    continueParams = False

            if continueParams:

                #Add to tally
                total+=1

                #Return on investment = new price / old price

                returnValue = (dataArray[j+timeout][0]-fee)/dataArray[j][0]
                totalReturn*=returnValue
                tradePercent+=returnValue

    if total==0:
        tradePercent=1    
    else:
        tradePercent=tradePercent/total
        
    #Return the average trade percent, the total number of times, and the parameters
    returnList = [tradePercent, totalReturn, total, fee, timeout, divisor] + [param for param in allParams for param in param]
    return returnList
    # print(tradePercent)
    # print(tradePerDay)

In [107]:
#Test algo to compile it
algo(np.array([[0, 50.00, 0],[10, 1.0, 3]]), weekDfs2[0][['open','close','time']].to_numpy())

[1.0002985066314023, 1.0072537504856045, 25.0, 0.0, 50.0, 0.0, 10.0, 1.0, 3.0]

In [5]:
#Params
#rough commission fee
fee=[0.005]

#change
changeList=np.arange(1.000,1.001,0.00005)

#timeout
timeoutList=[1,2,3,5,10,20,30,50]

#ema sma price before
emaTicksBeforeList=[20,50]
smaTicksBeforeList=[3,5,10,20,50]
#days before list
priceTicksBeforeList=[1,2,3,5,10,20,30,50]


priceParamProduct = list(product(priceTicksBeforeList, changeList, [1]))
#emaParamProduct = list(product(emaTicksBeforeList, changeList, [2]))
smaParamProduct = list(product(smaTicksBeforeList, changeList, [3]))
allParamProduct = priceParamProduct + smaParamProduct

allSettingsProduct = list(product(fee, timeoutList, fee))

depth = 2
xParamCombinations = list(combinations(allParamProduct, depth))

fullCombinations=[]
for a in allSettingsProduct:
    for b in xParamCombinations:
        fullCombinations.append(np.array(([a] + [c for c in b])))
len(fullCombinations)

269360

In [22]:
#Run the algo for each 2week period over the full combinations list with multiprocessing
weekResults=[]
for k in range(len(weekDfs2)):
    currentDataArray = weekDfs2[k][['open','close','time']].to_numpy()
    partialAlgo = partial(algo, dataArray=currentDataArray)

    #List of results to store in right order
    allResults=[]

    #Start multiprocessing
    with multiprocessing.Pool(8) as pool:
        for result in pool.map(
            partialAlgo,
            fullCombinations
        ):
            allResults.append(result)
    weekResults.append(allResults)

In [36]:
#Save to pickle
file_path = "../Results/SP500/V2/AAPL-minute2021-2Week-269kParams/"

for k in range(len(weekResults)):
    tempPath=file_path+str(k)+".pkl"
    with open(tempPath, "wb") as file:
        pickle.dump(weekResults[k], file)

In [8]:
#Load from pickle
weekResults=[]
file_path = "../Results/SP500/V2/AAPL-minute2021-2Week-269kParams/"

for k in range(69):
    tempPath=file_path+str(k)+".pkl"
    with open(tempPath, "rb") as file:
        weekResults.append(pickle.load(file))
    file.close()

In [99]:
#Minimum number of trades to be considered and sort and filter results, get best results
minTotal=5
maxTotal=10000000
bestWeekResults=[]

for k in range(len(weekResults)):
    filteredResults=[]
    for result in weekResults[k]:
        if maxTotal>=result[2]>=minTotal:
            filteredResults.append(result)

    #Sort results by each trade and by total
    sortedResults = sorted(filteredResults.copy(), reverse=True)
    bestResults = sortedResults[0:100]
    bestWeekResults.append(bestResults)

In [100]:
#Print best result from each period
for k in range(len(bestWeekResults)):
    print(bestWeekResults[k][0][0:2])

[1.0078200773545076, 1.0396775017389608]
[1.0045303780112838, 1.0228436518281245]
[1.0061064801563937, 1.0308824825779226]
[1.0043572736063555, 1.0219291629366356]
[1.0057392763959716, 1.0290162566774097]
[1.0041936694150542, 1.025379927054281]
[1.0049392185982122, 1.02491345287434]
[1.0037218214015573, 1.0263412226493398]
[1.005019494902472, 1.0304842927664448]
[1.0052573375867966, 1.026554074355301]
[1.005572680410474, 1.0281330838895566]
[1.006367889743903, 1.0322282877982896]
[1.004465209009361, 1.022493105536216]
[1.0054030234081606, 1.027295475906167]
[1.0047267004045008, 1.0238212144129533]
[1.003500860472517, 1.02834357512943]
[1.010543539971832, 1.0649475988508195]
[1.0082705744760727, 1.0419798643533191]
[1.006176540113806, 1.0312346844442484]
[1.0080791232406514, 1.0408481417539202]
[1.0051952156708472, 1.0261954369133555]
[1.0078188516217976, 1.0396355034949378]
[1.0108977516742872, 1.0555753281316682]
[1.0059130403891101, 1.0299157796130396]
[1.0067774935027172, 1.03427706

In [101]:
#Save best results to pickle

file_path = "../Results/SP500/V2/AAPL-minute2021-2Week-269kParams/min5best.pkl"

with open(file_path, "wb") as file:
    pickle.dump(bestWeekResults, file)


In [126]:
#Load best results from pickle
bestWeekResults=[]

tempPath = "../Results/SP500/V2/AAPL-minute2021-2Week-269kParams/min20best.pkl"

with open(tempPath, "rb") as file:
    bestWeekResults=(pickle.load(file))

In [127]:
#Check best results with next periods data
results=[]
for k in range(len(bestWeekResults)-1):
    total=0
    currentDataArray=weekDfs1[k*2+2][['open','close','time']].to_numpy()
    for j in range(10):
        array=np.array([bestWeekResults[k][j][3:6],bestWeekResults[k][j][6:9],bestWeekResults[k][j][9:12]])
        total+=algo(array, currentDataArray)[0]
    results.append(total/10)
for k in range(len(results)):
    print(k, results[k])
print("Average", sum(results)/len(results))

0 0.9992505642219298
1 1.0005886895600133
2 0.9957226538672996
3 1.0002513264978594
4 0.9998580759672153
5 1.0003075007413758
6 1.0001598356283834
7 1.0010742734672209
8 1.0006245214304683
9 0.9999165159493166
10 1.0006950698712604
11 1.0006065733748126
12 1.0001437120181165
13 1.0015538766584953
14 0.9999054707214056
15 1.0001394973539874
16 0.9987913959671497
17 0.9995864635900821
18 1.0009129163738408
19 1.0008292737298166
20 1.0003122502132666
21 0.997492939523924
22 1.0002695461285451
23 1.0009739433176563
24 0.998707758110904
25 1.0017135604881662
26 1.000474823058839
27 0.9990180934832884
28 0.9986270964612803
29 0.9965075676765316
30 1.0013340727857218
31 0.9991343229347633
32 0.9997171544064225
33 1.0013525932217142
34 1.0005601532205235
35 1.0004167668533919
36 1.0001629664515201
37 0.9999386550486555
38 0.9991865004395148
39 0.998485810295881
40 1.0008748568465817
41 0.9985119130781381
42 0.9993972701905042
43 1.0025427743305182
44 0.9985919687269099
45 1.0003948957733142
46

In [105]:
bestWeekResults[1][0]

[1.0045303780112838,
 1.0228436518281245,
 5.0,
 0.0,
 50.0,
 0.0,
 2.0,
 1.0006500000000014,
 1.0,
 10.0,
 1.000900000000002,
 3.0]